In [14]:
import GEOparse
import pandas as pd
import os

In [ ]:
gse = GEOparse.get_GEO(geo="GSE122126", destdir="./")

In [ ]:
# inspect a single sample
for gsm_name, gsm in gse.gsms.items():
    print("Name: ", gsm_name)
    print("Metadata:",)
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print ("Table data:",)
    print (gsm.table.head())
    break

In [4]:
# scan current directory to open the first directory
idat_folder = "idats"

In [ ]:
file_mapping = {}

for f in os.listdir(idat_folder):
    if f.endswith('.idat'):
        print("idat file found: ", f)
        file_mapping[f.split('_')[0]] = f
        

In [ ]:
sample_sheet = pd.DataFrame()

characteristics_identifier = "disease state"

for _, gsm in gse.gsms.items():
    gsm_name = gsm.name
    basket = {}
    basket['Sample_Name'] = ''
    basket['Sample_Plate'] = ''
    basket['Pool_ID'] = ''
    char_dict = {key: value for item in gsm.metadata['characteristics_ch1'] for key, value in [item.split(': ')]}
    basket['Sample_Group'] = char_dict[characteristics_identifier]
    if gsm_name in file_mapping:
        print(f"Found {gsm_name} in selected idat files")
        basket['Sentrix_ID'] = file_mapping[gsm_name].split('_')[1]
        basket['Sentrix_Position'] = file_mapping[gsm_name].split('_')[2]
        sample_sheet = pd.concat([sample_sheet, pd.DataFrame([basket])], ignore_index=True)
        
print(sample_sheet)


In [13]:
sample_sheet.sort_values(by=['Sample_Group', 'Sentrix_ID', 'Sentrix_Position'], inplace=True)
sample_sheet.reset_index(drop=True, inplace=True)
sample_sheet['Sample_Name'] = sample_sheet.index + 1
sample_sheet.to_csv("sample_sheet.csv", index=False)